In [11]:
import pandas as pd
from sklearn import preprocessing
from collections import defaultdict
from sklearn.impute import SimpleImputer

In [19]:
df = pd.read_excel('breast_cancer_data.xlsx', index_col='ehr')
#print(df.isnull().any())

birthDates = df.birth_date
for date in birthDates:
    print(pd.to_datetime('2022-12-18').year-pd.to_datetime(date).year)

print(birthDates)


55
72
69
72
45
71
64
72
73
76
81
69
45
62
81
65
67
42
79
52
61
70
51
79
83
63
68
75
69
55
68
67
79
64
62
54
62
76
50
59
81
67
53
81
87
78
39
57
68
58
61
69
43
71
60
82
84
37
60
51
79
56
46
60
106
50
52
49
44
72
66
51
53
61
50
57
48
73
67
70
71
67
56
43
52
56
73
97
69
50
50
58
54
59
62
49
76
62
53
74
48
44
55
41
54
80
59
68
64
89
78
60
38
61
64
72
48
58
51
62
52
66
56
53
48
53
81
46
49
69
80
60
42
79
51
56
53
67
82
67
56
50
47
52
74
47
50
60
64
62
66
74
50
50
50
77
55
58
51
54
59
49
101
43
46
53
69
55
52
50
56
52
58
50
57
45
43
51
39
53
65
52
43
78
69
53
77
70
97
76
42
47
85
52
52
60
47
48
51
54
81
83
59
50
79
79
49
42
46
53
82
54
35
82
49
ehr
6849    1967-08-08
268     1950-03-11
1458    1953-09-17
268     1950-03-11
2013    1977-08-19
           ...    
191     1940-02-07
6482    1968-02-21
2564    1987-04-19
2730    1940-02-02
2376    1973-05-04
Name: birth_date, Length: 215, dtype: object


We delete the NULL values by using the Simple Imputer

In [7]:
imp_cat = SimpleImputer(strategy='most_frequent')
columns = df.columns
index = df.index
df_cat = pd.DataFrame(imp_cat.fit_transform(df), columns=columns, index=index)

print(df_cat.isnull().any())

Unnamed: 0         False
side               False
neoadjuvant        False
grade              False
invasive           False
er_positive        False
pr_positive        False
her2_positive      False
ki67               False
birth_date         False
diagnosis_date     False
death_date         False
recurrence_year    False
menarche_age       False
menopause_age      False
pregnancy          False
abort              False
birth              False
caesarean          False
hist_type          False
dtype: bool


Now we are going to eliminate the labels that do not give relevant information but would increase the computational cost of our data treatments.
e.g: Columns with too many different values, such as descriptions or names.

Now we are about to subsitute the categorical labels that are represented by strings with numerical values, in order to avoid working with Strings.

In [8]:
ohe = preprocessing.OneHotEncoder(sparse=False)
df_cat_ohe = pd.DataFrame(ohe.fit_transform(df_cat), 
    columns=ohe.get_feature_names_out(df_cat.columns.tolist()), index=df_cat.index)

print(df_cat_ohe)

      Unnamed: 0_AA  Unnamed: 0_AB  Unnamed: 0_AC  Unnamed: 0_AD  \
ehr                                                                
6849            1.0            0.0            0.0            0.0   
268             0.0            1.0            0.0            0.0   
1458            0.0            0.0            1.0            0.0   
268             0.0            0.0            0.0            1.0   
2013            0.0            0.0            0.0            0.0   
...             ...            ...            ...            ...   
191             0.0            0.0            0.0            0.0   
6482            0.0            0.0            0.0            0.0   
2564            0.0            0.0            0.0            0.0   
2730            0.0            0.0            0.0            0.0   
2376            0.0            0.0            0.0            0.0   

      Unnamed: 0_AE  Unnamed: 0_AF  Unnamed: 0_AG  Unnamed: 0_AH  \
ehr                                            